In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!tar -xvzf "/content/drive/MyDrive/nlphilo/v1.0.rainbow.tar.gz" -C "/content/drive/MyDrive/nlphilo/"

rainbow/
rainbow/physicaliqa/
rainbow/physicaliqa/train.physicaliqa.csv
rainbow/physicaliqa/test.physicaliqa.csv
rainbow/physicaliqa/validation.physicaliqa.csv
rainbow/cosmosqa/
rainbow/cosmosqa/test.cosmosqa.csv
rainbow/cosmosqa/validation.cosmosqa.csv
rainbow/cosmosqa/train.cosmosqa.csv
rainbow/winogrande/
rainbow/winogrande/train_m.winogrande.csv
rainbow/winogrande/train.winogrande.csv
rainbow/winogrande/train_xl.winogrande.csv
rainbow/winogrande/train_s.winogrande.csv
rainbow/winogrande/test.winogrande.csv
rainbow/winogrande/train_l.winogrande.csv
rainbow/winogrande/train_xs.winogrande.csv
rainbow/winogrande/validation.winogrande.csv
rainbow/anli/
rainbow/anli/test.anli.csv
rainbow/anli/validation.anli.csv
rainbow/anli/train.anli.csv
rainbow/hellaswag/
rainbow/hellaswag/validation.hellaswag.csv
rainbow/hellaswag/test.hellaswag.csv
rainbow/hellaswag/train.hellaswag.csv
rainbow/socialiqa/
rainbow/socialiqa/validation.socialiqa.csv
rainbow/socialiqa/train.socialiqa.csv
rainbow/sociali

In [29]:
!cd /content/drive/MyDrive/nlphilo

In [4]:
!ls

drive  sample_data


In [18]:
!tar -xvzf "/content/drive/MyDrive/nlphilo/v1.0.rainbow.tar.gz.checksum"


gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [2]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5
import t5.models
import seqio

Installing dependencies...


In [3]:
!pip install torch
!pip install transformers
!pip install pytorch_lightning
import argparse
import glob
import json
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# attempt 1

In [2]:
!nvidia-smi

Thu Mar 24 17:21:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install sentencepiece

In [4]:
!pip install transformers
!pip install pytorch_lightning

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [8]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [9]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    self._hparams = hparams
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  """def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}"""

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    return {"val_loss": loss}
  
  """def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}"""

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  """def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, on_tpu = None,  
                     using_native_amp=None, using_lbfgs = None, optimizer_closure=None,):
    
    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()"""
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [ ]:
def get_dataset(tokenizer, type_path, args):
  return ImdbDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)

In [ ]:
# Namespace(adam_epsilon=1e-08, data_dir='aclImdb', early_stop_callback=False, eval_batch_size=8, fp_16=False, gradient_accumulation_steps=16, 
#           learning_rate=0.0003, max_grad_norm=1.0, max_seq_length=512, model_name_or_path='t5-3b', n_gpu=1, num_train_epochs=2, opt_level='O1', 
#           output_dir='t5_imdb_sentiment', seed=42, tokenizer_name_or_path='t5-3b', train_batch_size=8, warmup_steps=0, weight_decay=0.0)


In [10]:
model = torch.load("/content/drive/MyDrive/nlphilo/model.ckpt-1025000.data-00031-of-00032")

OverflowError: ignored

In [11]:
from keras.models import load_model
model = load_model("/content/drive/MyDrive/nlphilo/model.ckpt-1025000.data-00031-of-00032")

OSError: ignored

In [ ]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='t5-3b',
    tokenizer_name_or_path='t5-3b',
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

# attempt 2

In [13]:
# rainbow/
# rainbow/physicaliqa/
# rainbow/physicaliqa/train.physicaliqa.csv
# rainbow/physicaliqa/test.physicaliqa.csv
# rainbow/physicaliqa/validation.physicaliqa.csv
# rainbow/cosmosqa/
# rainbow/cosmosqa/test.cosmosqa.csv
# rainbow/cosmosqa/validation.cosmosqa.csv
# rainbow/cosmosqa/train.cosmosqa.csv
# rainbow/winogrande/
# rainbow/winogrande/train_m.winogrande.csv
# rainbow/winogrande/train.winogrande.csv
# rainbow/winogrande/train_xl.winogrande.csv
# rainbow/winogrande/train_s.winogrande.csv
# rainbow/winogrande/test.winogrande.csv
# rainbow/winogrande/train_l.winogrande.csv
# rainbow/winogrande/train_xs.winogrande.csv
# rainbow/winogrande/validation.winogrande.csv
# rainbow/anli/
# rainbow/anli/test.anli.csv
# rainbow/anli/validation.anli.csv
# rainbow/anli/train.anli.csv
# rainbow/hellaswag/
# rainbow/hellaswag/validation.hellaswag.csv
# rainbow/hellaswag/test.hellaswag.csv
# rainbow/hellaswag/train.hellaswag.csv
# rainbow/socialiqa/
# rainbow/socialiqa/validation.socialiqa.csv
# rainbow/socialiqa/train.socialiqa.csv
# rainbow/socialiqa/test.socialiqa.csv
df = pd.read_csv( "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/train.physicaliqa.csv", index_col="index")
df_validation = pd.read_csv( "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/validation.physicaliqa.csv", index_col="index")

df

,inputs,targets
index,,
0,"[physicaliqa]:\n<goal>When boiling butter, whe...",1
1,[physicaliqa]:\n<goal>To permanently attach me...,0
2,[physicaliqa]:\n<goal>how do you indent someth...,0
3,[physicaliqa]:\n<goal>how do you shake somethi...,0
4,[physicaliqa]:\n<goal>Clean tires</goal>\n<sol...,1
...,...,...
16108,[physicaliqa]:\n<goal>tack</goal>\n<sol1>can b...,1
16109,[physicaliqa]:\n<goal>How to clean a dishwashe...,1
16110,[physicaliqa]:\n<goal>To seal leather for furn...,1


## 2.1

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small" #"allenai/unifiedqa-t5-large" 
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [33]:
def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

In [34]:
for row in df["inputs"]:
  print(row, "\n", run_model(row), "\n", "\n", "\n")

[physicaliqa]:
<goal>When boiling butter, when it's ready, you can</goal>
<sol1>Pour it onto a plate</sol1>
<sol2>Pour it into a jar</sol2> 
 ["[physicaliqa]: goal>When boiling butter, when it'"] 
 
 

[physicaliqa]:
<goal>To permanently attach metal legs to a chair, you can</goal>
<sol1>Weld the metal together to get it to stay firmly in place</sol1>
<sol2>Nail the metal together to get it to stay firmly in place</sol2> 
 ['[physicaliqa]: sol2>Nail the metal together to'] 
 
 

[physicaliqa]:
<goal>how do you indent something?</goal>
<sol1>leave a space before starting the writing</sol1>
<sol2>press the spacebar</sol2> 
 ['[physicaliqa]: /goal> sol1>le'] 
 
 

[physicaliqa]:
<goal>how do you shake something?</goal>
<sol1>move it up and down and side to side quickly.</sol1>
<sol2>stir it very quickly.</sol2> 
 ['[physicaliqa]: /goal>how do you shake something?'] 
 
 

[physicaliqa]:
<goal>Clean tires</goal>
<sol1>Pour water, cape off caked on dirt. Use  speed wool to clean out crevices

KeyboardInterrupt: ignored

## 2.2

In [14]:

def find_question(text):
    r = re.compile(f'<goal>(.*?)</goal>')
    m = r.search(text["inputs"])
    if m:
        question = m.group(1)
    else: question = ""
    return question
    
def find_answer(text):
    val = text["targets"]+1
    r = re.compile(f'<sol{val}>(.*?)</sol{val}>')
    m = r.search(text["inputs"])
    if m:
        answer = m.group(1)
    else: answer = ""
    return answer


df["question"] = df.apply(find_question, axis=1)
df["answer"] = df.apply(find_answer, axis=1)
df_validation["question"] = df_validation.apply(find_question, axis=1)
df_validation["answer"] = df_validation.apply(find_answer, axis=1)
df_validation

,inputs,targets,question,answer
index,,,,
0,[physicaliqa]:\n<goal>How do I ready a guinea ...,0,How do I ready a guinea pig cage for it's new ...,Provide the guinea pig with a cage full of a f...
1,[physicaliqa]:\n<goal>dresser</goal>\n<sol1>re...,1,dresser,"finish, woodgrain with bobby pin"
2,[physicaliqa]:\n<goal>To fight Ivan Drago in R...,1,To fight Ivan Drago in Rocky for sega master s...,You have to defeat Apollo Creed and Clubber La...
3,[physicaliqa]:\n<goal>Make outdoor pillow.</go...,1,Make outdoor pillow.,Blow into trash bag and tie with rubber band.
4,[physicaliqa]:\n<goal>ice box</goal>\n<sol1>wi...,0,ice box,will turn into a cooler if you add water to it
...,...,...,...,...
1833,[physicaliqa]:\n<goal>To add chocolate and car...,1,To add chocolate and caramel to hot milk and c...,Chop the chocolate into small chunks. Bring th...
1834,[physicaliqa]:\n<goal>To make orange pulp from...,0,To make orange pulp from oranges.,Peel the oranges and put in blender.
1835,[physicaliqa]:\n<goal>Roast marshmallows.</goa...,1,Roast marshmallows.,Create a fire using wood. Poke a marshmallow w...


In [15]:
df.to_json("/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/train.json")
df_validation.to_json("/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/validate.json")

In [16]:
!gzip /content/drive/MyDrive/nlphilo/rainbow/physicaliqa/train.json
!gzip /content/drive/MyDrive/nlphilo/rainbow/physicaliqa/validate.json

In [4]:
import gzip
import json

DATA_DIR = ""
# Public directory of Natural Questions data on GCS.
NQ_JSONL_DIR = " "
NQ_SPLIT_FNAMES = {
    "train": "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/train.json.gz",
    "validation": "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/validate.json.gz"
}

nq_counts_path = os.path.join(DATA_DIR, "nq-counts.json")
nq_tsv_path = {
    "train": "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/train.json",#os.path.join(DATA_DIR, "physicaliqa/train.physicaliqa.csv"),
    "validation": "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/validate.json",#os.path.join(DATA_DIR, "physicaliqa/test.physicaliqa.csv")
}

# num_nq_examples = num_nq_examples = json.load(tf.io.gfile.GFile(nq_counts_path))
DEFAULT_OUTPUT_FEATURES = {
    "inputs":
        seqio.Feature(
            vocabulary=t5.data.get_default_vocabulary(), add_eos=True),
    "targets":
        seqio.Feature(
            vocabulary=t5.data.get_default_vocabulary(), add_eos=True)
}
def nq_jsonl_to_tsv(in_fname, out_fname):

  def extract_answer(tokens, span):
    """Reconstruct answer from token span and remove extra spaces."""
    start, end = span["start_token"], span["end_token"]
    ans = " ".join(tokens[start:end])
    # Remove incorrect spacing around punctuation.
    ans = ans.replace(" ,", ",").replace(" .", ".").replace(" %", "%")
    ans = ans.replace(" - ", "-").replace(" : ", ":").replace(" / ", "/")
    ans = ans.replace("( ", "(").replace(" )", ")")
    ans = ans.replace("`` ", "\"").replace(" ''", "\"")
    ans = ans.replace(" 's", "'s").replace("s ' ", "s' ")
    return ans

  count = 0
  with tf.io.gfile.GFile(in_fname, "rb") as infile,\
       tf.io.gfile.GFile(out_fname, "w") as outfile:
    for line in gzip.open(infile):
      ex = json.loads(line)
      # Remove any examples with more than one answer.
      for i in range(16113):
        s = str(i)
        # Questions in NQ do not include a question mark.
        if out_fname == "/content/drive/MyDrive/nlphilo/rainbow/physicaliqa/validate.json" and i == 1838: return count
        question = ex["question"][s] + "?"
        answer = ex["answer"][s]
        # Write this line as <question>\t<answer>
        outfile.write("%s\t%s\n" % (question, answer))
        count += 1
        tf.logging.log_every_n(
            tf.logging.INFO,
            "Wrote %d examples to %s." % (count, out_fname),
            1000)
      return count

if tf.io.gfile.exists(nq_counts_path):
  # Used cached data and counts.
  tf.logging.info("Loading NQ from cache.")
  num_nq_examples = json.load(tf.io.gfile.GFile(nq_counts_path))
else:
  # Create TSVs and get counts.
  tf.logging.info("Generating NQ TSVs.")
  num_nq_examples = {}
  for split, fname in NQ_SPLIT_FNAMES.items():
    num_nq_examples[split] = nq_jsonl_to_tsv(
        os.path.join(NQ_JSONL_DIR, fname), nq_tsv_path[split])
  json.dump(num_nq_examples, tf.io.gfile.GFile(nq_counts_path, "w"))

INFO:tensorflow:Loading NQ from cache.


In [5]:
def nq_dataset_fn(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  # Split each "<question>\t<answer>" example into (question, answer) tuple.
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # Map each tuple to a {"question": ... "answer": ...} dict.
  ds = ds.map(lambda *ex: dict(zip(["question", "answer"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_dataset_fn("train").take(5)):
  print(ex)

A few raw train examples...
{'question': b"When boiling butter, when it's ready, you can?", 'answer': b'Pour it into a jar'}
{'question': b'To permanently attach metal legs to a chair, you can?', 'answer': b'Weld the metal together to get it to stay firmly in place'}
{'question': b'how do you indent something??', 'answer': b'leave a space before starting the writing'}
{'question': b'how do you shake something??', 'answer': b'move it up and down and side to side quickly.'}
{'question': b'Clean tires?', 'answer': b'Pour water, scrape off caked on dirt. Use a steel wool to clean out crevices and narrow spaces.'}


In [6]:
def trivia_preprocessor(ds):
  def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text

  def to_inputs_and_targets(ex):
    """Map {"question": ..., "answer": ...}->{"inputs": ..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["trivia question: ", normalize_text(ex["question"])]),
        "targets": normalize_text(ex["answer"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [7]:
DEFAULT_OUTPUT_FEATURES = {
    "inputs":
        seqio.Feature(
            vocabulary=t5.data.get_default_vocabulary(), add_eos=True),
    "targets":
        seqio.Feature(
            vocabulary=t5.data.get_default_vocabulary(), add_eos=True)
}
seqio.TaskRegistry.remove("nq_context_free")
seqio.TaskRegistry.add(
    "nq_context_free",
    # Specify the task source.
    source=seqio.FunctionDataSource(
        # Supply a function which returns a tf.data.Dataset.
        dataset_fn=nq_dataset_fn,
        splits=["train", "validation"],
        # Not required, but helps for mixing and auto-caching.
        num_input_examples=num_nq_examples),
    # Supply a list of functions that preprocess the input tf.data.Dataset.
    preprocessors=[
        trivia_preprocessor,
        seqio.preprocessors.tokenize_and_append_eos,
    ],
    # Lowercase targets before computing metrics.
    postprocess_fn=t5.data.postprocessors.lower_text,
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    output_features=DEFAULT_OUTPUT_FEATURES,
)

In [8]:
nq_task = seqio.TaskRegistry.get("nq_context_free")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 128, "targets": 32})
print("A few preprocessed train examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)

A few preprocessed train examples...
{'inputs_pretokenized': b'trivia question: make my seams nearly invisible.?', 'inputs': array([22377,   822,    10,   143,    82, 13846,     7,  2111, 17472,
           5,    58,     1], dtype=int32), 'targets_pretokenized': b'a clear nylon thread will make seems nearly invisible.', 'targets': array([    3,     9,   964, 23903,  4546,    56,   143,  1330,  2111,
       17472,     5,     1], dtype=int32)}
{'inputs_pretokenized': b'trivia question: to dry clothes faster in a dryer.?', 'inputs': array([22377,   822,    10,    12,  2192,  4954,  3627,    16,     3,
           9, 15629,     5,    58,     1], dtype=int32), 'targets_pretokenized': b'throw a dry towel into the dryer with your wet clothes. the load will take almost half the time to dry.', 'targets': array([ 3793,     3,     9,  2192, 15580,   139,     8, 15629,    28,
          39,    62,    17,  4954,     5,     8,  4002,    56,   240,
         966,   985,     8,    97,    12,  2192,     5,

## model

In [14]:
MODEL_SIZE = "small" 
# Public GCS path for T5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "/content/drive/MyDrive/nlphilo/models/"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
TPU_TOPOLOGY = "v2-8"
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  TPU_ADDRESS = tpu.get_master()
  print('Running on TPU:', TPU_ADDRESS)
except ValueError:
  TPU_ADDRESS = ""
  # raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')


# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 16, 1),

    # "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

gpu_options = tf.GPUOptions(allow_growth=True)
# session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
# session.close()
tf.compat.v1.reset_default_graph()

In [10]:

# The models are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 128, "targets": 32},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=None,
    iterations_per_loop=100,
)

In [11]:
import time
from datetime import timedelta
start = time.time()

In [13]:

FINETUNE_STEPS = 10 #25000 
model.finetune(
    mixture_or_task_name="nq_context_free",
    pretrained_model_dir=PRETRAINED_DIR,
    finetune_steps=FINETUNE_STEPS)



ValueError: ignored

In [ ]:

end = time.time()
elapsed_time = end-start
print("elapsed time: " + str(timedelta(seconds=elapsed_time)))

In [45]:
!nvidia-smi 

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import random 

def print_random_predictions(task_name, n=10):
  """Print n predictions from the train split of a task."""
  # Grab the dataset for this task.
  ds = seqio.TaskRegistry.get(task_name).get_dataset(
      split="train",
      sequence_length={"inputs": 128, "targets": 32},
      shuffle=False)

  def _prediction_file_to_ckpt(path):
    """Extract the global step from a prediction filename."""
    return int(path.split("_")[-2])

  # Grab the paths of all logged predictions.
  prediction_files = tf.io.gfile.glob(
      os.path.join(
          MODEL_DIR,
          "train_eval/%s_*_predictions" % task_name))
  # Get most recent prediction file by sorting by their step.
  latest_prediction_file = sorted(
      prediction_files, key=_prediction_file_to_ckpt)[-1]

  # Collect (inputs, targets, prediction) from the dataset and predictions file
  results = []
  with tf.io.gfile.GFile(latest_prediction_file) as preds:
    for ex, pred in zip(tfds.as_numpy(ds), preds):
      results.append((tf.compat.as_text(ex["inputs_pretokenized"]),
                      tf.compat.as_text(ex["targets_pretokenized"]),
                      pred.strip()))

  print("<== Random predictions for %s using checkpoint %s ==>\n" %
        (task_name, 
         _prediction_file_to_ckpt(latest_prediction_file)))

  for inp, tgt, pred in random.choices(results, k=10):
    print("Input:", inp)
    print("Target:", tgt)
    print("Prediction:", pred)
    print("Counted as Correct?", tgt == pred)
    print()

print_random_predictions("nq_context_free")

In [ ]:
# First import the t5 mixtures so we can load the training mixture's vocabulary.
import t5.data.mixtures

question_1 = "Where is the Google headquarters located?" #@param {type:"string"}
question_2 = "What is the most populous country in the world?" #@param {type:"string"}
question_3 = "Who are the 4 members of The Beatles?" #@param {type:"string"}
question_4 = "How many teeth do humans have?" #@param {type:"string"}

questions = [question_1, question_2, question_3, question_4]

now = time.time()
# Write out the supplied questions to text files.
predict_inputs_path = os.path.join(MODEL_DIR, "predict_inputs_%d.txt" % now)
predict_outputs_path = os.path.join(MODEL_DIR, "predict_outputs_%d.txt" % now)
# Manually apply preprocessing by prepending "triviaqa question:".
with tf.io.gfile.GFile(predict_inputs_path, "w") as f:
  for q in questions:
    f.write("trivia question: %s\n" % q.lower())

In [ ]:
from contextlib import contextmanager

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

# Ignore any logging so that we only see the model's answers to the questions.
with tf_verbosity_level('ERROR'):
  model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
  model.predict(
      input_file=predict_inputs_path,
      output_file=predict_outputs_path,
      # Select the most probable output token at each step.
      temperature=0,
  )

# The output filename will have the checkpoint appended so we glob to get 
# the latest.
prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
with tf.io.gfile.GFile(prediction_files[-1]) as f:
  for q, a in zip(questions, f):
    if q:
      print("Q: " + q)
      print("A: " + a)
      print()

In [ ]:
export_dir = os.path.join(MODEL_DIR, "export")

model.batch_size = 1 # make one prediction per call
saved_model_path = model.export(
    export_dir,
    checkpoint_step=-1,  # use most recent
    beam_size=1,  # no beam search
    temperature=1.0,  # sample according to predicted distribution
)
print("Model saved to:", saved_model_path)
print(now, time.time(), now - time.time())

In [ ]:
import tensorflow as tf
import tensorflow_text  # Required to run exported model.

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["inputs"].name: x}
    )

predict_fn = load_predict_fn(saved_model_path)

In [ ]:
def answer(question):
  return predict_fn([question])[0].decode('utf-8')

for question in ["trivia question: where is the google headquarters?",
                 "trivia question: what is the most populous country in the world?",
                 "trivia question: who are the 4 members of the beatles?",
                 "trivia question: how many teeth do humans have?"]:
    print(answer(question))